# Transform ASK library

In [5]:
from datasets import load_dataset

dataset = load_dataset("ctu-aic/ask_library_cs", split="train")
dataset

Dataset({
    features: ['title', 'question', 'answer', 'okres', 'library', 'category', 'used_sources', 'date', 'url', 'id', 'language'],
    num_rows: 16273
})

## Korektor

In [15]:
#korektor
import requests

korektor_url = "https://lindat.mff.cuni.cz/services/korektor/api/correct"

def correct_text(text, model='czech-diacritics_generator-130202'):
    params = {
        'model': model,
        'data': text,
    }
    response = requests.post(korektor_url, data=params)
    if response.status_code != 200:
        raise Exception(f"Failed to correct text: {response.text}")
    return response.json()['result']



In [16]:
correct_text("Toto je testovací veta bez diakritiky.")

'Toto je testovací věta bez diakritiky.'

In [17]:
def correct_map(example):
    if example['language'] == 'cs':
        example['question_diacritics_korektor'] = correct_text(example['question'])
        example['answer_diacritics_korektor'] = correct_text(example['answer'])
        example['question_spell_korektor'] = correct_text(example['question'], model='czech-spellchecker-130202')
        example['answer_spell_korektor'] = correct_text(example['answer'], model='czech-spellchecker-130202')
    else:
        example['question_diacritics_korektor'] = ""
        example['answer_diacritics_korektor'] = ""
        example['question_spell_korektor'] = ""
        example['answer_spell_korektor'] = ""
    return example

In [18]:
dataset = dataset.map(correct_map)

Map:   0%|          | 0/16273 [00:00<?, ? examples/s]

Exception: Failed to correct text: <html>
<head><title>429 Too many requests</title></head>
<body bgcolor="white">
<center><h1>429 Too many requests</h1></center>
<center><h3>If you want to continue using our services, please contact us at this email address:</h3> <h2>lindat-help [at sign] ufal.mff.cuni.cz</h2></center>
</body>
</html>


## Other

In [2]:
#filter czech
dataset = dataset.filter(lambda example: example['language'] == 'cs')
dataset

Dataset({
    features: ['title', 'question', 'answer', 'okres', 'library', 'category', 'used_sources', 'date', 'url', 'id', 'language'],
    num_rows: 15789
})

In [3]:
dataset[895]

{'title': 'brát či dávat',
 'question': 'když mě svrbí dlaň říká se ,že budu brát, ale pravánebo levá, brát či dávat',
 'answer': 'Dobrá den, k Vašemu dotazu se nám podařilo dohledat následující informace : u Romů se dodnes traduje několik pověr ve spojení s penězi. V jedné z nich se říká, že když vás svrbí levá dlaň, znamená to, že v nejbližší době obdržíte nějaké peníze. A když vás naopak svrbí pravá dlaň, Romové říkají, že budete muset něco zaplatit. Levá bere, pravá dává. Nejsem pověrčivý, ale říká se, že jsou věci mezi nebem a zemí. U nás tomu věřila pouze matka: když si škrábala levou dlaň, říkala, že už brzy přijde listonoška a přinese jí peníze. My jsme se jen posmívali, ale vždy tomu tak bylo. Nikdy se nestalo, že by ji tato pověra zklamala. V životě jsem se několikrát přesvědčil, že znamení této pověry je pravdivé a dnes už jí věřím i já. Vždy si vzpomenu, jak nám matka vysvětlovala, že každé svrbění levé dlaně není stejné. Podle ní vás začne svrbět levá dlaň vždy dva až tři 

In [4]:
#filter all newline and whitespace characters
import re

def clean_text(example):
    #match all whitespace characters and replace them with a single space
    example['question'] = re.sub(r'\s+', ' ', example['question']).strip()
    example['answer'] = re.sub(r'\s+', ' ', example['answer']).strip()
    return example

dataset = dataset.map(clean_text)

In [27]:
correct_text("Jak se mas?")

<Response [200]>


'Jak se máš?'

In [15]:
FILTER_STRINGS = ["Děkuji za odpověď.", "Prosíme o laskavou odpověď", "Mnohokrát děkuji.", "děkuji", "Děkuji a velice se těším na odpověď."]

#strip the strings!

#diacritics restoration ufal bert/czaccent/korektor ufal/GECCC finetuned ufal

#filter out containing links -> too few -> another link filtering?

#look at the data without these greetings
FILTER_GREETING = ["Vážený paní", "Dobrý den", "Vážený pane", "Vážený čtenáři", "Vážená paní", "Dobré odpoledne"]
dataset_g = dataset.filter(lambda x: not any(greeting in x['answer'] for greeting in FILTER_GREETING) and not any(greeting.lower() in x['answer'] for greeting in FILTER_STRINGS))
dataset_g

Filter:   0%|          | 0/15789 [00:00<?, ? examples/s]

Dataset({
    features: ['title', 'question', 'answer', 'okres', 'library', 'category', 'used_sources', 'date', 'url', 'id', 'language'],
    num_rows: 2282
})

In [16]:
dataset_g[202]

{'title': 'ČERNÝ, Jan. Knieha lékarska, kteráž slove herbář aneb Zelinář',
 'question': 'Dobrý den, hledám informace o knize Knieha lekarska Kteraz slowe herbarz, Mikuláše Klaudiána, z roku 1517. Za jakoukoliv zprávu předem děkuji.',
 'answer': 'Dobry den, pravdepodobne mate na mysli herbar Jana Cerneho: CERNY, Jan (Niger). Knieha lekarska kteraz slowe herbarz: aneb zelinarz :welmi vziteczna: z mnohych knieh latinskych. y zskutecznych praczij wybrana:poczina se sstiastnie... Norimberk, 1517. Podrobny bibliograficky zaznam knihy vcetne informaci o knihovnach, ktere herbar ve svych sbirkach vlastni, naleznete v databazi Knihopis Digital -http://db.knihopis.org/l.dll?cll~1758, soucasti zaznamu jsou i nahledy nekterych stranek -http://db.knihopis.org/l.dll?cli~1758&TL=. Pokud byste mel zajem, muzete si napr. v NK CR vypujcit kriticke vydani Kniehy lekarske z roku 1981. Podminkou je platny ctenarsky prukaz NK CR (http://www.nkp.cz/pages/page.php3?nazev=Jak_to_tady_chodi&submenu3=98),knihu s

In [11]:
import re
import nltk
nltk.download('punkt_tab')

def remove_url(text:str) -> str:
    url_in_brackets_pattern = re.compile(r'\s*\([^)(]*https?:\/\/[^\s()]+[^)(]*\)')
    if "http" in text:
        #remove urls in brackets
        text = re.sub(url_in_brackets_pattern, '', text)

        #if url still in the answer
        if "http" in text:
            #handle standalone urls
            ##obtain sentences
            sentences=nltk.tokenize.sent_tokenize(text, language="czech")
            ##find sentences without urls
            filtered_sentences = [sentence for sentence in sentences if "http" not in sentence]
            
            text = " ".join(filtered_sentences)

    
    return text

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/mlynatom/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [18]:
dataset_g["answer"][202]

'Dobry den, pravdepodobne mate na mysli herbar Jana Cerneho: CERNY, Jan (Niger). Knieha lekarska kteraz slowe herbarz: aneb zelinarz :welmi vziteczna: z mnohych knieh latinskych. y zskutecznych praczij wybrana:poczina se sstiastnie... Norimberk, 1517. Podrobny bibliograficky zaznam knihy vcetne informaci o knihovnach, ktere herbar ve svych sbirkach vlastni, naleznete v databazi Knihopis Digital -http://db.knihopis.org/l.dll?cll~1758, soucasti zaznamu jsou i nahledy nekterych stranek -http://db.knihopis.org/l.dll?cli~1758&TL=. Pokud byste mel zajem, muzete si napr. v NK CR vypujcit kriticke vydani Kniehy lekarske z roku 1981. Podminkou je platny ctenarsky prukaz NK CR (http://www.nkp.cz/pages/page.php3?nazev=Jak_to_tady_chodi&submenu3=98),knihu si muzete objednat prostrednictvim Elektronickeho katalogu NK CR - baze NKC (http://sigma.nkp.cz/cze/nkc). Soucasti knihy je uvod, ktery herbar blize charakerizuje. ČERNÝ, Jan. Knieha lékarska, kteráž slove herbář aneb Zelinář. Praha : Avicenum, 

In [19]:
remove_url(dataset_g["answer"][202])

'Dobry den, pravdepodobne mate na mysli herbar Jana Cerneho: CERNY, Jan (Niger). Knieha lekarska kteraz slowe herbarz: aneb zelinarz :welmi vziteczna: z mnohych knieh latinskych. y zskutecznych praczij wybrana:poczina se sstiastnie... Norimberk, 1517. Pokud byste mel zajem, muzete si napr. v NK CR vypujcit kriticke vydani Kniehy lekarske z roku 1981. Podminkou je platny ctenarsky prukaz NK CR,knihu si muzete objednat prostrednictvim Elektronickeho katalogu NK CR - baze NKC. Soucasti knihy je uvod, ktery herbar blize charakerizuje. ČERNÝ, Jan. Knieha lékarska, kteráž slove herbář aneb Zelinář. Praha : Avicenum, 1981. 463 s. Resume: Prvni kriticke vydani zakladniho spisu ceskeho humanistickeho lekare, ktery poprve vysel v Norimberce r. 1517. Cerneho cesky psany herbar se vyznacuje kritickym pristupme k dane problematice a snahou o usporadani tehdejsiho chaotickeho botanickeho nazvoslovi. Cerny dava kazde rostline jediny cesky nazev a podrobnym popisem umoznuje jeji presne urceni. Herbar 